In [ ]:
# !pip install selenium
# !apt-get update 
# !apt install chromium-chromedriver

# from selenium import webdriver
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')
# driver = webdriver.Chrome('chromedriver',options=chrome_options)

In [9]:
import time
import csv
import re

import numpy as np
import pandas as pd
import requests
import bs4
import lxml.etree as xml

import pprint

import pycountry

from bs4 import BeautifulSoup
from googlesearch import search

from tqdm.auto import tqdm
tqdm.pandas()

import warnings
warnings.filterwarnings("ignore")

In [4]:
def get_country(txt):
    for c in pycountry.countries:
        if c.name in txt:
            return c.name
    return ''

In [5]:
CLEANR = re.compile('<.*?>')

def cleanhtml(raw_html):
  cleantext = re.sub(CLEANR, '\n', str(raw_html))
  cleantext = ' '.join([str(x).strip() for x in cleantext.split('\n')])
  return cleantext

In [6]:
def name_convert(self):
    searchval = 'cnbc stocks '+self
    link = []
    for url in search(searchval, lang='en', num_results=5):
        link.append(url)
    links = [x if 'www.cnbc.com/quotes/' in x else '' for x in link]
    tickers = []
    for link in links:
        try:
            link = str(link)
            link = link.split("/")
            if link[-1]=='':
                ticker=link[-2]
            else:
                x=link[-1].split('=')
                ticker=x[-1]
            tickers.append(ticker)
        except:
            continue
    return list(set(tickers))

In [7]:
companies = ["Harley-Davidson Inc. (Ticker: HOG)",
"Amtek Auto Ltd",
"Renault SA (Ticker: RNO)",
"Continental AG (Ticker: CON)",
"The Kroger Co. (Ticker: KR)",
]

companies = {k:v for k,v in enumerate(companies)}

companies = pd.DataFrame(companies.items(), columns=['id','company'])

companies.head()

,id,company
0,0,Harley-Davidson Inc. (Ticker: HOG)
1,1,Amtek Auto Ltd
2,2,Renault SA (Ticker: RNO)
3,3,Continental AG (Ticker: CON)
4,4,The Kroger Co. (Ticker: KR)


In [10]:
companies['tickers'] = companies.company.progress_apply(name_convert)
companies['found'] = companies.tickers.apply(lambda x: len(x)>0)
companies

100%|██████████| 5/5 [00:10<00:00,  2.10s/it]


,id,company,tickers,found
0,0,Harley-Davidson Inc. (Ticker: HOG),"[HOG, HAR-FF]",True
1,1,Amtek Auto Ltd,"[LUMAXTECH-IN, METALFORGE-IN]",True
2,2,Renault SA (Ticker: RNO),"[9974.T-JP, 0IB0.L, RNL-FF, 52C-FF, PROFK-GR, ...",True
3,3,Continental AG (Ticker: CON),"[NIVINFRA-IN, ALSIM-FR, SANGHVIMOV-IN, I1TW3'-...",True
4,4,The Kroger Co. (Ticker: KR),"[KR, 0JS2-GB]",True


In [11]:
def cnbc_extract(tickers=None):
    if isinstance(tickers, str):
        tickers = [tickers]
    data = []
    for ticker in tickers:
        try:
            info_lst = []
            url = 'https://www.cnbc.com/quotes/{}?tab=profile'.format(ticker)
            r = requests.get(url, headers ={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'})
            soup = BeautifulSoup(r.text, 'html.parser')
            for info in soup.find_all('span', {'class': 'QuoteStrip-symbolAndExchange'}):
                info_lst.append(info.text.split(':')[1])
            url2 = 'https://apps.cnbc.com/view.asp?symbol={}&uid=stocks/summary'.format(ticker)
            r = requests.get(url2, headers ={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'})
            soup = BeautifulSoup(r.text, 'html.parser')
            for info in soup.find_all('span', {'class': 'drkr1'})[0]:
                info_lst.append(info)
            url3 = 'https://apps.cnbc.com/view.asp?symbol={}&uid=stocks/stockvsPeers'.format(ticker)
            r = requests.get(url3, headers ={'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'})
            soup = BeautifulSoup(r.text, 'html.parser')
            for info in soup.find_all('div', {'id': 'companyHeader'}):
                info_lst.append([x for x in info.text.split('\r\n') if 'Industry:' in x][0].replace('Industry:','').strip())
            data.append(info_lst)
        except Exception as e:
            continue
    return data

In [12]:
companies['data'] = companies.tickers.progress_apply(cnbc_extract)

100%|██████████| 5/5 [01:33<00:00, 18.71s/it]


In [13]:
companies

,id,company,tickers,found,data
0,0,Harley-Davidson Inc. (Ticker: HOG),"[HOG, HAR-FF]",True,"[[NYSE, United States, Recreational Vehicles &..."
1,1,Amtek Auto Ltd,"[LUMAXTECH-IN, METALFORGE-IN]",True,"[[National Stock Exchange of India, India, Aut..."
2,2,Renault SA (Ticker: RNO),"[9974.T-JP, 0IB0.L, RNL-FF, 52C-FF, PROFK-GR, ...",True,"[[France, --], [Frankfurt Stock Exchange, Fran..."
3,3,Continental AG (Ticker: CON),"[NIVINFRA-IN, ALSIM-FR, SANGHVIMOV-IN, I1TW3'-...",True,"[[Euronext Paris, United Kingdom, Software], [..."
4,4,The Kroger Co. (Ticker: KR),"[KR, 0JS2-GB]",True,"[[NYSE, United States, Food Retailers & Wholes..."


In [19]:
_df = pd.DataFrame(companies.data.tolist())
_df.columns = ['data_{}'.format(x) for x in range(_df.shape[1])]
_df = _df[['data_0', 'data_1', 'data_3']]
_df2 = pd.concat([companies, _df], axis=1)
_df2.fillna('', inplace=True)


def pandas_list_explode(df, col, prefix=None, drop_original=True, collist=[]):
    if prefix is None:
        prefix = col+'_'
    df[col] = df[col].apply(lambda x: '///'.join(x))
    _df = df[col].str.split('///', expand=True).add_prefix(prefix)
    _df.columns = [prefix+x for x in collist]
    df = df.join(_df)
    if drop_original:
        df.drop(col, axis=1, inplace=True)
    return df

collist = ['exchange','country','industry']
for col in _df.columns:
    _df2 = pandas_list_explode(_df2, col=col, collist=collist)

_df2.fillna('', inplace=True)
_df2

,id,company,tickers,found,data,data_0_exchange,data_0_country,data_0_industry,data_1_exchange,data_1_country,data_1_industry,data_3_exchange,data_3_country,data_3_industry
0,0,Harley-Davidson Inc. (Ticker: HOG),"[HOG, HAR-FF]",True,"[[NYSE, United States, Recreational Vehicles &...",NYSE,United States,Recreational Vehicles & Boats,Frankfurt Stock Exchange,United States,--,,,
1,1,Amtek Auto Ltd,"[LUMAXTECH-IN, METALFORGE-IN]",True,"[[National Stock Exchange of India, India, Aut...",National Stock Exchange of India,India,Auto Parts,National Stock Exchange of India,India,Commercial Vehicles & Parts,,,
2,2,Renault SA (Ticker: RNO),"[9974.T-JP, 0IB0.L, RNL-FF, 52C-FF, PROFK-GR, ...",True,"[[France, --], [Frankfurt Stock Exchange, Fran...",France,--,,Frankfurt Stock Exchange,France,--,Euronext Paris,France,Automobiles
3,3,Continental AG (Ticker: CON),"[NIVINFRA-IN, ALSIM-FR, SANGHVIMOV-IN, I1TW3'-...",True,"[[Euronext Paris, United Kingdom, Software], [...",Euronext Paris,United Kingdom,Software,National Stock Exchange of India,India,Commercial Vehicle-equipment Leasing,,,
4,4,The Kroger Co. (Ticker: KR),"[KR, 0JS2-GB]",True,"[[NYSE, United States, Food Retailers & Wholes...",NYSE,United States,Food Retailers & Wholesalers,,,,,,


In [20]:
%run ~/postrun.ipynb

Author: Sparsh A.

Last updated: 2022-12-01 16:32:04

Compiler    : Clang 10.0.0 
OS          : Darwin
Release     : 21.6.0
Machine     : x86_64
Processor   : i386
CPU cores   : 4
Architecture: 64bit

numpy    : 1.20.3
csv      : 1.0
requests : 2.28.1
bs4      : 4.11.1
pycountry: 22.3.5
lxml     : 4.8.0
re       : 2.2.1
pandas   : 1.4.3
sys      : 3.9.7 (default, Sep 16 2021, 08:50:36) 
[Clang 10.0.0 ]

